### Batch Inference for Tensorflow Model we Just Discussed.

we need the following structure 
```
├── code
│   ├── inference.py
│   └── requirements.txt
└── yourModel
    ├── saved_model.pb
    └── variables
       ├── variables.data-00000-of-00001
       └── variables.index
```
we will be zipping up all of this

In [1]:
!aws s3 cp s3://sagemaker-us-east-1-973994155064/soundclass-2020-12-27-17-16-41-996/output/model.tar.gz ./export/

download: s3://sagemaker-us-east-1-973994155064/soundclass-2020-12-27-17-16-41-996/output/model.tar.gz to export/model.tar.gz


In [2]:
!tar -xvzf ./export/model.tar.gz -C ./export/

000000001/
000000001/assets/
000000001/saved_model.pb
000000001/variables/
000000001/variables/variables.index
000000001/variables/variables.data-00000-of-00001


In [13]:
%cd export

/root/VKOSound_classification/Batch_Inference/export


In [16]:
! rm -r code/.ipynb_checkpoints/

In [20]:
!tar -czvf model.tar.gz code 1

code/
code/requirements.txt
code/inference.py
1/
1/variables/
1/variables/variables.data-00000-of-00001
1/variables/variables.index
1/saved_model.pb
1/assets/


In [25]:
import os
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
sm_role=sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
# See the following document for more on SageMaker Roles:
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html
role = sm_role
bucket = 'feidemo'
prefix = 'envsoundata'
s3_path = 's3://{}/{}'.format(bucket, prefix)

model_data = sagemaker_session.upload_data('model.tar.gz',
                                           bucket,
                                           os.path.join(prefix, 'model'))
                                           
tensorflow_serving_model = TensorFlowModel(model_data=model_data,
                                 role=role,
                                 framework_version='2.3.1',
                                 sagemaker_session=sagemaker_session)

In [27]:
output_path = 's3://feidemo/envsoundata/output'
tensorflow_serving_transformer = tensorflow_serving_model.transformer(
                                     instance_count=2,
                                     instance_type='ml.p3.2xlarge',
                                     max_concurrent_transforms=64,
                                     max_payload=1,
                                     output_path=output_path)

input_path = 's3://feidemo/envsoundata/data/test'
tensorflow_serving_transformer.transform(input_path, content_type='application/x-image')

...................................INFO:__main__:starting services
INFO:tfs_utils:using default model name: model
INFO:tfs_utils:tensorflow serving model config: 
model_config_list: {
  config: {
    name: "model",
    base_path: "/opt/ml/model",
    model_platform: "tensorflow"
  }
}


INFO:__main__:using default model name: model
INFO:__main__:tensorflow serving model config: 
model_config_list: {
  config: {
    name: "model",
    base_path: "/opt/ml/model",
    model_platform: "tensorflow"
  }
}


INFO:__main__:tensorflow version info:
TensorFlow ModelServer: 2.3.0-rc0+dev.sha.no_git
TensorFlow Library: 2.3.0
INFO:__main__:tensorflow serving command: tensorflow_model_server --port=10000 --rest_api_port=10001 --model_config_file=/sagemaker/model-config.cfg --max_num_load_retries=0 
INFO:__main__:started tensorflow serving (pid: 11)
INFO:__main__:nginx config: 
load_module modules/ngx_http_js_module.so;

worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr er